In [1]:
import pandas as pd
import numpy as np

In [2]:
file = "pkl/uber-raw-data-combined.csv.pkl"
df = pd.read_pickle(file)
df

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512
...,...,...,...,...
4534322,9/30/2014 22:57:00,40.7668,-73.9845,B02764
4534323,9/30/2014 22:57:00,40.6911,-74.1773,B02764
4534324,9/30/2014 22:58:00,40.8519,-73.9319,B02764
4534325,9/30/2014 22:58:00,40.7081,-74.0066,B02764


In [3]:
def time_sampling(start, end):
    df["Date/Time"] = pd.to_datetime(df["Date/Time"])
    filter_df = df[(df["Date/Time"] >= start) & (df["Date/Time"] <= end)]
    sample_df = filter_df.sort_values(by="Date/Time")
    return sample_df

### Inputs go by:
"Year-Month-Day Hour:Minute:Second"

*time is in 24 hour clock

the rows are out of order because original csv was sorted by base values, not time.

In [4]:
#time_sampling("2014-04-01 00:00:00", "2014-04-29 00:00:00")

In [5]:
if (df["Date/Time"].dtypes != np.datetime64):
    df["Date/Time"] = pd.to_datetime(df["Date/Time"])
print(f"Max: {df.max()}")
print(f"Min: {df.min()}")
lat = df["Lat"]
lon = df["Lon"]
time = df["Date/Time"]

Max: Date/Time    2014-09-30 22:59:00
Lat                      42.1166
Lon                     -72.0666
Base                      B02764
dtype: object
Min: Date/Time    2014-04-01 00:00:00
Lat                      39.6569
Lon                      -74.929
Base                      B02512
dtype: object


In [14]:
def bucket(lat, lon, time, lat_bucket_size, lon_bucket_size, time_bucket_period):
    df["lat_bucket"] = (lat // lat_bucket_size) * lat_bucket_size
    df["lon_bucket"] = (lon // lon_bucket_size) * lon_bucket_size
    df["time_bucket"] = time.dt.to_period(time_bucket_period).dt.start_time
    matrix = df.groupby([df["time_bucket"], df["lat_bucket"], df["lon_bucket"]]).size().reset_index(name="count")

    return matrix

In [17]:
bucket(lat, lon, time, 0.1, 0.1, "min")
#s, min, h, d, w, m, y

,time_bucket,lat_bucket,lon_bucket,count
0,2014-04-01 00:00:00,40.7,-74.0,3
1,2014-04-01 00:01:00,40.7,-74.0,1
2,2014-04-01 00:02:00,40.7,-74.0,3
3,2014-04-01 00:03:00,40.7,-74.0,1
4,2014-04-01 00:03:00,40.7,-73.9,1
...,...,...,...,...
1032098,2014-09-30 22:58:00,40.8,-74.0,1
1032099,2014-09-30 22:59:00,40.6,-74.0,2
1032100,2014-09-30 22:59:00,40.6,-73.8,1
1032101,2014-09-30 22:59:00,40.7,-74.1,2


In [ ]:
def cluster(df, bin):
    tf = pd.cut(df, bins=bin)
    count = tf.value_counts().sort_index() #datatype = pandas series
    print(count) #.index[0] for range, .iloc[0] for value